# 02 - NRRD, MHD, and Analyze I/O

This notebook demonstrates how to use the `diffusemri` library for Input/Output (I/O) operations with NRRD (Nearly Raw Raster Data), MHD (MetaImage Header), and Analyze 7.5 file formats. We will cover:

1.  Creating dummy files in these formats.
2.  Reading data from these formats using `diffusemri` functions.
3.  Writing data to these formats using `diffusemri` functions.
4.  Converting these formats to and from NIfTI using the library's utilities (conceptually via CLI calls, demonstrated with Python function calls to the CLI interface for notebook compatibility).
5.  Important considerations for handling DWI (Diffusion-Weighted Imaging) metadata and format limitations.

In [ ]:
import os
import shutil
import numpy as np
import nibabel as nib
import nrrd # For creating dummy NRRD directly
import SimpleITK as sitk # For creating dummy MHD directly

# diffusemri library imports
from data_io.nrrd_utils import read_nrrd_data, write_nrrd_data
from data_io.mhd_utils import read_mhd_data, write_mhd_data
from data_io.analyze_utils import read_analyze_data, write_analyze_data

# For demonstrating CLI functionality via Python calls
# This assumes cli.run_format_conversion.main can accept a list of string arguments
from cli.run_format_conversion import main as conversion_cli_main 

# Setup a temporary directory for example files
TEMP_DIR = "temp_io_formats_example"
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
os.makedirs(TEMP_DIR)

print(f"Temporary directory for examples: {os.path.abspath(TEMP_DIR)}")

## Part 1: NRRD (.nrrd, .nhdr)

NRRD is a flexible format that can store 2D, 3D, or higher-dimensional image data along with extensive metadata in its header.

### Creating a Dummy NRRD File

In [ ]:
dummy_nrrd_data_np = np.arange(2*3*4, dtype=np.uint16).reshape((2, 3, 4)) # X, Y, Z
dummy_nrrd_filepath = os.path.join(TEMP_DIR, "dummy_anat.nrrd")

# Define NRRD header. For NIfTI compatibility, 'space' should ideally be LPS or RAS.
# 'space directions' define the columns of the affine matrix (excluding translation).
# 'space origin' defines the translation part.
nrrd_header = {
    'space': 'left-posterior-superior', # LPS
    'space directions': [[1.0, 0, 0], [0, 2.0, 0], [0, 0, 3.0]], # Voxel sizes for x, y, z
    'space origin': [10.0, 11.0, 12.0]
}

try:
    nrrd.write(dummy_nrrd_filepath, dummy_nrrd_data_np, header=nrrd_header)
    print(f"Created dummy NRRD: {dummy_nrrd_filepath}")
except Exception as e:
    print(f"Error creating dummy NRRD: {e}")

### Reading NRRD Data

In [ ]:
if os.path.exists(dummy_nrrd_filepath):
    image_data_nrrd, affine_nrrd, bvals_nrrd, bvecs_nrrd, header_nrrd = read_nrrd_data(dummy_nrrd_filepath)
    print("--- Reading NRRD ---")
    print(f"Data shape: {image_data_nrrd.shape}, dtype: {image_data_nrrd.dtype}")
    print(f"Affine matrix:\n{affine_nrrd}")
    print(f"b-values: {bvals_nrrd}") # Expected to be None for this anatomical example
    print(f"Header keys (sample): {list(header_nrrd.keys())[:5]}")
else:
    print(f"Dummy NRRD file {dummy_nrrd_filepath} not found for reading.")

### Writing NRRD Data

In [ ]:
new_nrrd_data_np = np.random.rand(3, 4, 5).astype(np.float32)
new_nrrd_affine = np.array([
    [2.0, 0,   0,   5.0],
    [0,   2.0, 0,   6.0],
    [0,   0,   2.5, 7.0],
    [0,   0,   0,   1.0]
])
output_nrrd_filepath = os.path.join(TEMP_DIR, "output_anat.nrrd")

try:
    write_nrrd_data(output_nrrd_filepath, new_nrrd_data_np, new_nrrd_affine)
    print(f"Successfully wrote NRRD file: {output_nrrd_filepath}")
    # Verify by reading it back
    r_data, r_affine, _, _, _ = read_nrrd_data(output_nrrd_filepath)
    assert np.allclose(r_data, new_nrrd_data_np), "Data mismatch after writing/reading NRRD"
    assert np.allclose(r_affine, new_nrrd_affine), "Affine mismatch after writing/reading NRRD"
    print("Verification by reading back successful.")
except Exception as e:
    print(f"Error writing or verifying NRRD: {e}")

### NRRD <-> NIfTI Conversion (using CLI via Python calls)

The `run_format_conversion.py` script provides CLI tools for these conversions. We can invoke its `main` function with appropriate arguments.

In [ ]:
# NRRD to NIfTI
output_nifti_from_nrrd = os.path.join(TEMP_DIR, "converted_from.nrrd.nii.gz")
nrrd2nii_args = [
    'nrrd2nii',
    '--input_nrrd', dummy_nrrd_filepath,
    '--output_nifti', output_nifti_from_nrrd
]
print(f"\nRunning NRRD to NIfTI: {' '.join(nrrd2nii_args)}")
try:
    conversion_cli_main(nrrd2nii_args)
    if os.path.exists(output_nifti_from_nrrd):
        print(f"NRRD to NIfTI conversion successful: {output_nifti_from_nrrd}")
    else:
        print("NRRD to NIfTI conversion failed (output file not found).")
except SystemExit as e:
    print(f"CLI call for nrrd2nii exited with code {e.code}")

# NIfTI to NRRD
# First, create a dummy NIfTI file to convert
dummy_nifti_data_for_nrrd = np.arange(3*3*3, dtype=np.int16).reshape((3,3,3))
dummy_nifti_affine_for_nrrd = np.diag([1.5, 1.5, 3.0, 1.0])
dummy_nifti_filepath_for_nrrd = os.path.join(TEMP_DIR, "temp_for_nrrd_conversion.nii.gz")
nib.save(nib.Nifti1Image(dummy_nifti_data_for_nrrd, dummy_nifti_affine_for_nrrd), dummy_nifti_filepath_for_nrrd)

output_nrrd_from_nifti = os.path.join(TEMP_DIR, "converted_from.nii.nrrd")
nii2nrrd_args = [
    'nii2nrrd',
    '--input_nifti', dummy_nifti_filepath_for_nrrd,
    '--output_nrrd', output_nrrd_from_nifti
]
print(f"\nRunning NIfTI to NRRD: {' '.join(nii2nrrd_args)}")
try:
    conversion_cli_main(nii2nrrd_args)
    if os.path.exists(output_nrrd_from_nifti):
        print(f"NIfTI to NRRD conversion successful: {output_nrrd_from_nifti}")
    else:
        print("NIfTI to NRRD conversion failed (output file not found).")
except SystemExit as e:
    print(f"CLI call for nii2nrrd exited with code {e.code}")

## Part 2: MHD (.mhd with .raw/.zraw or .mha)

MHD/MHA is another format often used in medical imaging, particularly with ITK-based software like 3D Slicer.

In [ ]:
dummy_mhd_data_np = np.arange(3*4*5, dtype=np.float32).reshape((5, 4, 3)) # SITK uses Z,Y,X order for numpy array
dummy_mhd_filepath = os.path.join(TEMP_DIR, "dummy_anat.mhd")

sitk_image = sitk.GetImageFromArray(dummy_mhd_data_np) # Z, Y, X
sitk_image.SetSpacing([1.1, 1.2, 1.3]) # X, Y, Z spacing
sitk_image.SetOrigin([5.0, 6.0, 7.0]) # X, Y, Z origin
# For simplicity, using identity direction cosine matrix (aligned with axes)
sitk_image.SetDirection(np.eye(3).flatten().tolist())

try:
    sitk.WriteImage(sitk_image, dummy_mhd_filepath)
    print(f"Created dummy MHD: {dummy_mhd_filepath}")
except Exception as e:
    print(f"Error creating dummy MHD: {e}")

### Reading MHD Data

In [ ]:
if os.path.exists(dummy_mhd_filepath):
    image_data_mhd, affine_mhd, bvals_mhd, bvecs_mhd, header_mhd = read_mhd_data(dummy_mhd_filepath)
    print("--- Reading MHD ---")
    # Note: read_mhd_data transposes to X,Y,Z order for data array
    print(f"Data shape: {image_data_mhd.shape}, dtype: {image_data_mhd.dtype}")
    print(f"Affine matrix:\n{affine_mhd}")
    print(f"b-values: {bvals_mhd}") # Expected to be None
    print(f"Header keys (sample): {list(header_mhd.keys())[:5]}")
else:
    print(f"Dummy MHD file {dummy_mhd_filepath} not found for reading.")

### Writing MHD Data

In [ ]:
new_mhd_data_np = (np.random.rand(4, 5, 6) * 255).astype(np.uint8) # X, Y, Z
new_mhd_affine = np.array([
    [0.5, 0,   0,   -10.0],
    [0,   0.5, 0,   -12.0],
    [0,   0,   0.8, -15.0],
    [0,   0,   0,   1.0]
])
output_mhd_filepath = os.path.join(TEMP_DIR, "output_anat.mha") # Using .mha for single file

try:
    write_mhd_data(output_mhd_filepath, new_mhd_data_np, new_mhd_affine)
    print(f"Successfully wrote MHD file: {output_mhd_filepath}")
    # Verify by reading it back
    r_data_mhd, r_affine_mhd, _, _, _ = read_mhd_data(output_mhd_filepath)
    assert np.allclose(r_data_mhd, new_mhd_data_np), "Data mismatch after writing/reading MHD"
    assert np.allclose(r_affine_mhd, new_mhd_affine), "Affine mismatch after writing/reading MHD"
    print("Verification by reading back MHD successful.")
except Exception as e:
    print(f"Error writing or verifying MHD: {e}")

### MHD <-> NIfTI Conversion (using CLI via Python calls)

In [ ]:
# MHD to NIfTI
output_nifti_from_mhd = os.path.join(TEMP_DIR, "converted_from.mhd.nii.gz")
mhd2nii_args = [
    'mhd2nii',
    '--input_mhd', dummy_mhd_filepath,
    '--output_nifti', output_nifti_from_mhd
]
print(f"\nRunning MHD to NIfTI: {' '.join(mhd2nii_args)}")
try:
    conversion_cli_main(mhd2nii_args)
    if os.path.exists(output_nifti_from_mhd):
        print(f"MHD to NIfTI conversion successful: {output_nifti_from_mhd}")
    else:
        print("MHD to NIfTI conversion failed.")
except SystemExit as e:
    print(f"CLI call for mhd2nii exited with code {e.code}")

# NIfTI to MHD
dummy_nifti_filepath_for_mhd = os.path.join(TEMP_DIR, "temp_for_mhd_conversion.nii.gz")
# Re-use the NIfTI created for NRRD example, or create a new one
if not os.path.exists(dummy_nifti_filepath_for_mhd):
    nib.save(nib.Nifti1Image(np.zeros((2,2,2)), np.eye(4)), dummy_nifti_filepath_for_mhd)

output_mhd_from_nifti = os.path.join(TEMP_DIR, "converted_from.nii.mha")
nii2mhd_args = [
    'nii2mhd',
    '--input_nifti', dummy_nifti_filepath_for_mhd,
    '--output_mhd', output_mhd_from_nifti
]
print(f"\nRunning NIfTI to MHD: {' '.join(nii2mhd_args)}")
try:
    conversion_cli_main(nii2mhd_args)
    if os.path.exists(output_mhd_from_nifti):
        print(f"NIfTI to MHD conversion successful: {output_mhd_from_nifti}")
    else:
        print("NIfTI to MHD conversion failed.")
except SystemExit as e:
    print(f"CLI call for nii2mhd exited with code {e.code}")

## Part 3: Analyze 7.5 (.hdr, .img)

Analyze 7.5 is an older format but still encountered occasionally. It has significant limitations regarding orientation information.

In [ ]:
dummy_analyze_data_np = np.arange(4*5*6, dtype=np.int16).reshape((4,5,6)) # X,Y,Z
dummy_analyze_affine = np.diag([-2.0, 2.0, 2.5, 1.0]) # Example with negative X determinant
dummy_analyze_filepath = os.path.join(TEMP_DIR, "dummy_analyze.hdr")

analyze_image = nib.AnalyzeImage(dummy_analyze_data_np, dummy_analyze_affine)
try:
    nib.save(analyze_image, dummy_analyze_filepath)
    print(f"Created dummy Analyze: {dummy_analyze_filepath} (and .img)")
except Exception as e:
    print(f"Error creating dummy Analyze: {e}")

### Reading Analyze Data

In [ ]:
if os.path.exists(dummy_analyze_filepath):
    image_data_analyze, affine_analyze, header_analyze = read_analyze_data(dummy_analyze_filepath)
    print("--- Reading Analyze ---")
    print(f"Data shape: {image_data_analyze.shape}, dtype: {image_data_analyze.dtype}")
    print(f"Affine matrix:\n{affine_analyze}")
    print(f"Header (AnalyzeHeader object): {type(header_analyze)}")
else:
    print(f"Dummy Analyze file {dummy_analyze_filepath} not found for reading.")

### Writing Analyze Data

In [ ]:
new_analyze_data_np = (np.random.rand(5,6,7) * 1000).astype(np.int16)
new_analyze_affine = np.array([
    [1.0, 0,   0,   20.0],
    [0,   1.0, 0,   22.0],
    [0,   0,   1.5, 25.0],
    [0,   0,   0,   1.0]
])
output_analyze_filepath = os.path.join(TEMP_DIR, "output_anat.hdr")

try:
    write_analyze_data(output_analyze_filepath, new_analyze_data_np, new_analyze_affine)
    print(f"Successfully wrote Analyze file: {output_analyze_filepath}")
    # Verify by reading it back
    r_data_analyze, r_affine_analyze, _ = read_analyze_data(output_analyze_filepath)
    assert np.allclose(r_data_analyze, new_analyze_data_np), "Data mismatch after writing/reading Analyze"
    # Affine comparison for Analyze can be tricky due to its limitations. 
    # Nibabel does its best to represent it.
    # print(f"Original affine for write:\n{new_analyze_affine}")
    # print(f"Affine after read:\n{r_affine_analyze}")
    # For simple cases, it should be close. With flips/reorientations, it can differ more.
    assert np.allclose(r_affine_analyze, new_analyze_affine, atol=1e-5), "Affine mismatch after writing/reading Analyze"
    print("Verification by reading back Analyze successful (data and affine). ")
except Exception as e:
    print(f"Error writing or verifying Analyze: {e}")

### Analyze <-> NIfTI Conversion (using CLI via Python calls)

In [ ]:
# Analyze to NIfTI
output_nifti_from_analyze = os.path.join(TEMP_DIR, "converted_from.analyze.nii.gz")
analyze2nii_args = [
    'analyze2nii',
    '--input_analyze', dummy_analyze_filepath,
    '--output_nifti', output_nifti_from_analyze
]
print(f"\nRunning Analyze to NIfTI: {' '.join(analyze2nii_args)}")
try:
    conversion_cli_main(analyze2nii_args)
    if os.path.exists(output_nifti_from_analyze):
        print(f"Analyze to NIfTI conversion successful: {output_nifti_from_analyze}")
    else:
        print("Analyze to NIfTI conversion failed.")
except SystemExit as e:
    print(f"CLI call for analyze2nii exited with code {e.code}")

# NIfTI to Analyze
dummy_nifti_filepath_for_analyze = os.path.join(TEMP_DIR, "temp_for_analyze_conversion.nii.gz")
if not os.path.exists(dummy_nifti_filepath_for_analyze):
     nib.save(nib.Nifti1Image(np.zeros((2,2,2), dtype=np.int16), np.eye(4)), dummy_nifti_filepath_for_analyze)

output_analyze_from_nifti = os.path.join(TEMP_DIR, "converted_from.nii.hdr")
nii2analyze_args = [
    'nii2analyze',
    '--input_nifti', dummy_nifti_filepath_for_analyze,
    '--output_analyze', output_analyze_from_nifti
]
print(f"\nRunning NIfTI to Analyze: {' '.join(nii2analyze_args)}")
try:
    conversion_cli_main(nii2analyze_args)
    if os.path.exists(output_analyze_from_nifti):
        print(f"NIfTI to Analyze conversion successful: {output_analyze_from_nifti}")
    else:
        print("NIfTI to Analyze conversion failed.")
except SystemExit as e:
    print(f"CLI call for nii2analyze exited with code {e.code}")

### Important Note on Analyze 7.5 Format Limitations

The Analyze 7.5 format (`.hdr`/`.img`) is an older format with significant limitations compared to NIfTI:
*   **Orientation Information:** It has very poor support for storing image orientation and coordinate system information. Affine transformations derived from Analyze headers can often be ambiguous or incorrect, especially if the image is not aligned with standard axial/coronal/sagittal planes. SForm/QForm information, which NIfTI uses for precise orientation, is absent.
*   **Metadata:** Analyze 7.5 has minimal capacity for storing rich metadata. Crucially for diffusion MRI, it **cannot natively store b-values or b-vectors**.
*   **Data Types:** Supports a more restricted set of data types than NIfTI.

**Consequence:** Converting data (especially DWI) to Analyze 7.5 format will likely result in the loss of critical orientation and DWI metadata. When converting from Analyze 7.5 to NIfTI, the resulting NIfTI's orientation should be carefully checked.

## Notes on DWI Data for NRRD/MHD

Both NRRD and MHD formats can store DWI metadata (b-values, b-vectors) using custom key-value pairs in their headers. The `diffusemri` library's `read_nrrd_data`/`read_mhd_data` functions attempt to parse common conventions for these fields, and `write_nrrd_data`/`write_mhd_data` can write them.

When using the Python functions directly, you can provide `bvals` and `bvecs` NumPy arrays to the `write_*_data` functions. When reading, these will be returned if found and parsed from the header.

In [ ]:
# Example: Writing DWI data to NRRD
dwi_nrrd_data = np.random.rand(10, 10, 5, 6).astype(np.float32) # X,Y,Z,NumGradients
dwi_nrrd_affine = np.eye(4)
dwi_bvals = np.array([0, 1000, 1000, 1000, 2000, 2000])
dwi_bvecs = np.random.rand(6, 3)
dwi_bvecs[0,:] = 0 # b0 vector
dwi_bvecs[1:,:] = dwi_bvecs[1:,:] / np.linalg.norm(dwi_bvecs[1:,:], axis=1, keepdims=True) # Normalize

dwi_output_nrrd_filepath = os.path.join(TEMP_DIR, "dwi_example.nrrd")
try:
    write_nrrd_data(dwi_output_nrrd_filepath, dwi_nrrd_data, dwi_nrrd_affine, 
                    bvals=dwi_bvals, bvecs=dwi_bvecs,
                    custom_fields={'source_study': 'Example DWI'})
    print(f"Successfully wrote DWI NRRD: {dwi_output_nrrd_filepath}")

    # Verify reading DWI NRRD
    img_data, aff, bvals_read, bvecs_read, header = read_nrrd_data(dwi_output_nrrd_filepath)
    assert np.allclose(img_data, dwi_nrrd_data), "DWI NRRD data mismatch"
    assert np.allclose(bvals_read, dwi_bvals), "DWI NRRD bvals mismatch"
    # Note: b-vector comparison might need care due to potential reorientation logic in read/write
    # For this direct write/read with identity affine, they should be very close.
    assert np.allclose(bvecs_read, dwi_bvecs, atol=1e-6), "DWI NRRD bvecs mismatch"
    assert header.get('source_study') == 'Example DWI', "Custom field missing/incorrect"
    print("DWI NRRD write/read verification successful.")
except Exception as e:
    print(f"Error with DWI NRRD example: {e}")

# Similar principles apply for MHD DWI data using write_mhd_data and read_mhd_data.

## Cleanup

Remove the temporary directory and its contents.

In [ ]:
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
    print(f"Cleaned up temporary directory: {TEMP_DIR}")